# Consumer sentiment and recessions

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import altair as alt
import altair_stiles as altstiles
import pandas as pd
import numpy as np

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

#### Read monthly index of consumer sentiment from University of Michigan's [survey of consumers](http://www.sca.isr.umich.edu/)

In [3]:
src = pd.read_csv(
    "http://www.sca.isr.umich.edu/files/tbmics.csv",
    names=["month", "year", "index"],
    header=0,
)

#### Create a date for timeseries

In [4]:
src["date"] = pd.to_datetime(src["month"].astype(str) + " " + src["year"].astype(str))

In [5]:
df = src.copy()

In [6]:
df.head()

,month,year,index,date
0,November,1952,86.2,1952-11-01
1,February,1953,90.7,1953-02-01
2,August,1953,80.8,1953-08-01
3,November,1953,80.7,1953-11-01
4,February,1954,82.0,1954-02-01


----

#### Recessions

In [7]:
wiki_src = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_recessions_in_the_United_States"
)[2]

In [8]:
wiki_src.columns = (
    wiki_src.columns.str.lower()
    .str.replace(" ", "_")
    .str.replace("_(peak_to_trough)", "", regex=False)
)

In [9]:
wiki_slim = wiki_src[["name", "period_range", "gdp_decline"]].copy()

In [10]:
wiki_slim[["start", "end"]] = (
    wiki_slim["period_range"].str.split("[").str[0].str.split("–", expand=True)
)

In [11]:
wiki_slim["gdp_decline"] = (
    wiki_slim["gdp_decline"]
    .str.split("[")
    .str[0]
    .str.replace("%", "", regex=False)
    .str.replace("−", "", regex=False)
    .astype(float)
) * -1

In [12]:
wiki_slim.drop(["period_range"], axis=1, inplace=True)

In [13]:
wiki_slim["start"] = pd.to_datetime(wiki_slim["start"])
wiki_slim["end"] = pd.to_datetime(wiki_slim["end"])

In [14]:
wiki_slim["start_year"] = wiki_slim["start"].dt.year

In [15]:
wiki_slim["type"] = "Recessions"

In [16]:
wiki_slim["gdp_decline_pos"] = wiki_slim["gdp_decline"] * -1

---

## Charts

#### Consumers and recessions

In [17]:
lines = (
    alt.Chart(df)
    .mark_line(size=2)
    .encode(
        x=alt.X("date", title="", axis=alt.Axis(tickCount=7, format="%Y")),
        y=alt.Y("index", axis=alt.Axis(tickCount=6), title=" "),
    )
    .properties(
        title="Index of consumer sentiment and recessions",
    )
)

points = (
    alt.Chart(df.query("date == date.max()"))
    .mark_point(color="black")
    .encode(
        x=alt.X("date", title="", axis=alt.Axis(tickCount=7, format="%Y")),
        y=alt.Y("index", axis=alt.Axis(tickCount=6), title=" "),
    )
)

text = (
    alt.Chart(df.query("date == date.max()"))
    .mark_text(dy=15, color="black")
    .encode(
        x=alt.X("date", title="", axis=alt.Axis(tickCount=7, format="%Y")),
        y=alt.Y("index", axis=alt.Axis(tickCount=6), title=" "),
        text=alt.Text("index:Q"),
    )
)

rect = (
    alt.Chart(wiki_slim[4:])
    .mark_rect(color="#e6e6e6")
    .encode(
        x="start:T",
        x2="end:T",
    )
)

rect + lines + points + text

alt.LayerChart(...)

#### Recessions

In [18]:
bubbles = (
    alt.Chart(wiki_slim)
    .mark_circle(color="#00d4d8")
    .encode(
        alt.X("start_year:O", axis=alt.Axis(), title=" "),
        alt.Y("type:N", title=" "),
        alt.Size(
            "gdp_decline_pos:Q",
            scale=alt.Scale(range=[0, 3000]),
            legend=None,
        ),
    )
    .properties(width=650, height=120)
)

text = (
    alt.Chart(wiki_slim)
    .mark_text(dy=-40, color="black")
    .encode(
        x=alt.X("start_year:O"),
        y=alt.Y("type:N"),
        text=alt.Text("gdp_decline:Q", format=""),
    )
)

text2 = (
    alt.Chart(wiki_slim.query("name=='Great Depression' | name=='COVID-19 recession'"))
    .mark_text(dy=40, color="black")
    .encode(
        x=alt.X("start_year:O"),
        y=alt.Y("type:N"),
        text=alt.Text("name"),
    )
)

(bubbles + text + text2).properties(
    title="U.S. recessions by start year and GDP decline"
)

alt.LayerChart(...)

---

In [19]:
df.to_csv("data/processed/consumer_sentiment_1952_current.csv", index=False)

In [20]:
wiki_slim.to_csv("data/processed/recessions_1929_current.csv", index=False)